In [21]:
# Import necessary libraries and load environment variables
import os
import gradio as gr
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [22]:
# Define configuration constants and global variables
DATA_PATH = "dataset_1/"
PERSIST_DIRECTORY = "chroma_db"
LLM_MODEL = "gpt-4o-mini"

SYSTEM_PROMPT = """"You are an expert chatbot specializing in comparing the economies of developing and developed countries.

Use the retrieved context to cite **specific numerical data** such as GDP growth rates, FDI inflows, CO2 emissions, and similar indicators. 

Do NOT generalize or summarize without including data points. 
If multiple values are retrieved, compare them explicitly (e.g., "GDP growth was 5.9% in India vs 2.1% in the USA").

Always include file names or sources when mentioning data."""

# Global variable for RAG chain
rag_chain = None

In [23]:
# Function to load documents from the dataset directory
def load_documents():
    print("\n[LOAD DOCS] Checking dataset path...")
    if not Path(DATA_PATH).exists():
        raise FileNotFoundError(f"Dataset directory '{DATA_PATH}' not found")
    
    print(f"[LOAD DOCS] Path exists. Searching for .txt files in {DATA_PATH}")
    loader = DirectoryLoader(DATA_PATH, glob="*.txt", loader_cls=TextLoader)
    documents = loader.load()
    
    if not documents:
        raise ValueError(f"No .txt files found in '{DATA_PATH}'")
    
    print(f"[LOAD DOCS] ✓ Successfully loaded {len(documents)} document(s)")
    for i, doc in enumerate(documents, 1):
        print(f"  - Document {i}: {doc.metadata.get('source', 'unknown')} ({len(doc.page_content)} chars)")
    
    return documents

In [24]:
# Function to chunk documents into smaller pieces
def chunk_documents(documents):
    print("\n[CHUNK] Starting document chunking...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)
    
    print(f"[CHUNK] ✓ Created {len(chunks)} chunks")
    total_chars = sum(len(chunk.page_content) for chunk in chunks)
    print(f"[CHUNK] Total characters in chunks: {total_chars}")
    
    return chunks

In [25]:
# Function to create and persist a vector store
def create_vector_store(chunks):
    print("\n[EMBED] Creating embeddings and vector store...")
    print("[EMBED] Initializing OpenAI embeddings...")
    
    embeddings = OpenAIEmbeddings()
    
    print(f"[EMBED] Embedding {len(chunks)} chunks into vectors...")
    vector_store = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    
    print("[EMBED] Persisting vector store to disk...")
    vector_store.persist()
    
    print(f"[EMBED] ✓ Vector store created and persisted to {PERSIST_DIRECTORY}")
    
    return vector_store

In [26]:
# Function to initialize the RAG chain
def initialize_rag_chain():
    global rag_chain
    
    print("\n" + "="*60)
    print("INITIALIZING RAG CHAIN")
    print("="*60)
    
    # Check if vector store exists
    if os.path.exists(PERSIST_DIRECTORY):
        print(f"\n[INIT] Found existing Chroma database at {PERSIST_DIRECTORY}")
        print("[INIT] Loading existing vector store...")
        vector_store = Chroma(
            persist_directory=PERSIST_DIRECTORY,
            embedding_function=OpenAIEmbeddings()
        )
        print("[INIT] ✓ Vector store loaded")
    else:
        print(f"\n[INIT] No existing Chroma database found at {PERSIST_DIRECTORY}")
        print("[INIT] Creating new vector store from documents...")
        documents = load_documents()
        chunks = chunk_documents(documents)
        vector_store = create_vector_store(chunks)
    
    # Initialize retriever
    print("\n[RETRIEVER] Setting up retriever with k=5...")
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    print("[RETRIEVER] ✓ Retriever initialized")
    
    # Test retriever
    print("\n[TEST] Testing retriever with sample query...")
    test_query = "developing countries health economic indicators"
    test_results = retriever.get_relevant_documents(test_query)
    print(f"[TEST] Query: '{test_query}'")
    print(f"[TEST] ✓ Retrieved {len(test_results)} chunks")
    if test_results:
        print(f"[TEST] First chunk preview: {test_results[0].page_content[:150]}...")
    
    # Initialize LLM
    print("\n[LLM] Initializing ChatOpenAI...")
    llm = ChatOpenAI(model=LLM_MODEL, temperature=0.3)
    print("[LLM] ✓ LLM initialized")
    
    # Create prompt
    print("\n[PROMPT] Creating prompt template...")
    prompt = ChatPromptTemplate.from_template(
        f"""{SYSTEM_PROMPT}

Context from documents:
{{context}}

Question: {{question}}"""
    )
    print("[PROMPT] ✓ Prompt template created")
    
    # Create RAG chain
    print("\n[CHAIN] Building RAG chain...")
    def format_docs(docs):
        formatted = "\n\n".join(doc.page_content for doc in docs)
        print(f"[CHAIN] Formatted {len(docs)} documents into context ({len(formatted)} chars)")
        return formatted
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    print("[CHAIN] ✓ RAG chain built successfully!")
    print("\n" + "="*60)
    print("RAG CHAIN INITIALIZATION COMPLETE")
    print("="*60 + "\n")

In [27]:
# CELL 6: Chat Function
def chat(message, history):
    try:
        print(f"\n[CHAT] User query: '{message}'")
        # Pass message as a simple string, not a dict
        response = rag_chain.invoke(message)
        print(f"[CHAT] Response generated ({len(response)} chars)")
        return response
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(f"[CHAT] ERROR: {error_msg}")
        return error_msg

In [28]:
# Set up the Gradio chat interface
iface = gr.ChatInterface(
    fn=chat,
    title="Economics Chatbot: Developing vs Developed Countries",
    description="Ask me anything about the economies of developing and developed countries.",
    theme=gr.themes.Soft(),
    examples=[
        "What are the main differences in GDP growth between developing and developed countries?",
        "How do renewable energy adoption rates differ between developed and developing nations?",
        "Why do developing countries have higher population growth rates?",
        "What role does FDI play in economic development?"
    ]
)

C:\ProgramData\anaconda3\envs\llms\Lib\site-packages\gradio\chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [29]:
# Main execution block to initialize and launch the application
if __name__ == "__main__":
    initialize_rag_chain()
    iface.launch(share=False)


INITIALIZING RAG CHAIN

[INIT] Found existing Chroma database at chroma_db
[INIT] Loading existing vector store...
[INIT] ✓ Vector store loaded

[RETRIEVER] Setting up retriever with k=5...
[RETRIEVER] ✓ Retriever initialized

[TEST] Testing retriever with sample query...
[TEST] Query: 'developing countries health economic indicators'
[TEST] ✓ Retrieved 5 chunks
[TEST] First chunk preview: Economic and Health Indicators: Developing vs Developed Countries (2000-2023)
Data Overview
This dataset contains 288 country-year observations compar...

[LLM] Initializing ChatOpenAI...
[LLM] ✓ LLM initialized

[PROMPT] Creating prompt template...
[PROMPT] ✓ Prompt template created

[CHAIN] Building RAG chain...
[CHAIN] ✓ RAG chain built successfully!

RAG CHAIN INITIALIZATION COMPLETE

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.



[CHAT] User query: 'Why do developing countries have higher population growth rates?'
[CHAIN] Formatted 5 documents into context (2573 chars)
[CHAT] Response generated (1140 chars)

[CHAT] User query: 'what is the Government Health Expenditure (% of GDP) of developed countries?'
[CHAIN] Formatted 5 documents into context (3188 chars)
[CHAT] Response generated (554 chars)
